## 제미나이 키로 확인하는 것

In [ ]:
# import json
# import os
# import google.generativeai as genai
# from pydantic import BaseModel, Field, ConfigDict
# from dotenv import load_dotenv # 로컬 환경에서 .env 사용
# from typing import Optional # Optional 타입 사용을 위해 추가
# import sys # sys 모듈 임포트 추가

# # ----------------------------------------------------------------------
# # 1. 파일 경로 설정 (★★★★ 이 부분을 당신의 실제 환경에 맞게 수정해야 합니다 ★★★★)
# #    - config.env와 JSON 데이터 파일들이 위치한 디렉토리의 절대 경로를 정확히 입력하세요.
# # ----------------------------------------------------------------------

# # config.env 파일의 절대 경로
# # 예시: macOS 사용자라면 "/Users/YourUsername/Desktop/Eagle_eye_gds/config.env"
# # 예시: Windows 사용자라면 "C:\\Users\\YourUsername\\Desktop\\Eagle_eye_gds\\config.env"
# # ★★★★ 여기에 실제 config.env 파일의 절대 경로를 입력해주세요 ★★★★
# CONFIG_ENV_PATH = "/Users/10fingers/Desktop/Eagle_eye_gds/config.env" # << 이 줄을 사용자 환경에 맞게 수정

# # JSON 데이터 파일들이 위치한 디렉토리의 절대 경로 (config.env와 동일한 디렉토리로 가정)
# # ★★★★ 여기에 실제 JSON 파일들이 있는 디렉토리의 절대 경로를 입력해주세요 ★★★★
# JSON_DATA_DIR = "/Users/10fingers/Desktop/Eagle_eye_gds/Score" # << 이 줄을 사용자 환경에 맞게 수정


# # .env 파일에서 환경 변수를 로드합니다.
# load_dotenv(dotenv_path=CONFIG_ENV_PATH)

# # 환경 변수에서 API 키를 로드
# API_KEY = os.getenv("GOOGLE_API_KEY")
# if not API_KEY:
#     print(f"오류: 환경 변수 'GOOGLE_API_KEY'가 설정되지 않았습니다. '{CONFIG_ENV_PATH}' 파일을 확인해주세요.")
#     sys.exit(1) # sys.exit()를 사용하여 프로그램 확실히 종료

# # Gemini 클라이언트 초기화
# genai.configure(api_key=API_KEY)

# # ----------------------------------------------------------------------
# # API 키 유효성 검증 함수 (옵션이지만 추천)
# # ----------------------------------------------------------------------
# def _check_api_key_validity():
#     """
#     설정된 Gemini API 키의 유효성을 간단한 API 호출로 검증합니다.
#     """
#     try:
#         # 모델 목록을 가져오는 간단한 API 호출을 시도합니다.
#         # 이 호출은 API 키가 유효하지 않으면 예외를 발생시킵니다.
#         list(genai.list_models())
#         print("API 키가 성공적으로 검증되었습니다. Gemini API에 접근할 수 있습니다.")
#         return True
#     except Exception as e:
#         print(f"오류: API 키 검증 실패. API 키가 유효하지 않거나 네트워크 문제가 있을 수 있습니다.")
#         print(f"상세 오류: {e}")
#         print("API 키를 다시 확인하거나, Google AI Studio에서 새 키를 발급받아보세요.")
#         return False

# # API 키 유효성 검증 실행
# if not _check_api_key_validity():
#     sys.exit(1) # API 키가 유효하지 않으면 프로그램 종료
# # ----------------------------------------------------------------------


# # 1. 매핑 및 예시 JSON 파일 로드 함수 (절대 경로를 받도록 수정)
# def load_json_data(full_file_path: str): # 인자 이름을 full_file_path로 변경하여 명확화, 타입 힌트 추가
#     """지정된 절대 경로의 JSON 파일을 읽어 Python 객체로 반환합니다."""
#     try:
#         with open(full_file_path, 'r', encoding='utf-8') as f:
#             return json.load(f)
#     except FileNotFoundError:
#         print(f"오류: 파일 '{full_file_path}'를 찾을 수 없습니다. 파일 경로를 다시 확인해주세요.")
#         return None
#     except json.JSONDecodeError:
#         print(f"오류: 파일 '{full_file_path}'의 JSON 형식이 올바르지 않습니다. 파일 내용을 확인해주세요.")
#         return None
#     except Exception as e:
#         print(f"오류: 파일 '{full_file_path}' 로드 중 예상치 못한 오류 발생: {e}")
#         return None

# # Pydantic 모델 정의 (★★ 'title' 필드 추가 및 exclude=True 설정 ★★)
# class StoreCategoryResponse(BaseModel):
#     model_config = ConfigDict(extra="ignore")


#     naver_id: str = Field(description="입력된 매장 정보의 고유 ID")
#     name: str = Field(description="입력된 매장 이름")
#     대분류: str = Field(description="매장 정보에서 추론된 대분류 카테고리")
#     중분류: str = Field(description="매장 정보에서 추론된 중분류 카테고리")
#     소분류: str = Field(description="매장 정보에서 추론된 소분류 카테고리 (해당 없으면 빈 문자열)")
#     메뉴_라벨: str = Field(description="매장 정보에서 추론된 라벨 (해당 없으면 빈 문자열)")
#     메뉴_점수: str = Field(description="추론된 라벨에 해당하는 점수 (해당 없으면 빈 문자열)")
#     메뉴_추론근거: str = Field(description="LLM이 해당 카테고리 및 라벨을 선택한 상세한 추론 과정")
   

# # 2. System Prompt 정의
# SYSTEM_PROMPT = """
# 당신은 대한민국 내 음식점 및 상점 카테고리 분류 전문가입니다.
# 주어진 매장 정보를 바탕으로 가장 정확한 '대분류', '중분류', '소분류', '라벨' 그리고 '점수'를 매핑해야 합니다.
# 매핑할 '라벨'이 제공된 '라벨 매핑 정보'에 존재한다면 반드시 해당 '점수'를 부여하세요.
# 만약 '라벨'이 존재하지 않아 '점수'를 부여할 수 없다면 빈 문자열('')로 표시하세요.
# 만약 해당되는 '소분류'나 '라벨'을 찾을 수 없다면 해당 필드는 빈 문자열('')로 표시하세요.
# 응답은 반드시 JSON 형식으로만 제공해야 합니다.
# 각 필드(`대분류`, `중분류`, `소분류`, `라벨`, `점수`, `추론근거`)는 문자열 타입이어야 합니다.
# '추론근거' 필드에 추론 과정을 상세히 설명해야 합니다.
# 카테고리 및 라벨 매핑 정보가 주어지지 않았다면, 주어진 텍스트 내용만을 기반으로 최선의 추론을 하되, 정보 부족을 '추론근거'에 명시하세요.
# """.strip()

# # 3. Few-Shot Examples 정의
# FEW_SHOT_EXAMPLES = f"""
# ## Few-Shot Examples (예시 학습)

# ### 2.1. Example Category Mappings (카테고리 매핑 예시)
# 다음은 음식 카테고리 매핑 정보의 예시입니다:
# 대분류: 음식점, 중분류: 한식, 소분류: 한정식, 목록: ["한정식"]
# 대분류: 음식점, 중분류: 일식, 소분류: 초밥,롤, 목록: ["초밥,롤"]
# 대분류: 음식점, 중분류: 양식, 소분류: 햄버거, 목록: ["햄버거"]
# 대분류: 음식점, 중분류: 중식, 소분류: 마라탕, 목록: ["마라탕"]
# 대분류: 음식점, 중분류: 술집, 소분류: 바(BAR), 목록: ["바(BAR)", "라운지바", "루프톱바"]

# ### 2.2. Example Score Mappings (점수 매핑 예시)
# 다음은 특정 음식점 라벨에 대한 정보의 예시입니다. 각 라벨에는 점수가 부여되어 있습니다 (5가 가장 높음, 1이 가장 낮음):
# 라벨: 수제 햄버거 전문점, 점수: 5, 매핑 카테고리: 햄버거, 관련 키워드: 햄버거
# 라벨: 샤브샤브, 점수: 5, 매핑 카테고리: 샤브샤브, 관련 키워드:
# 라벨: 갈비 무한 리필, 점수: 5, 매핑 카테고리: 고기뷔페,육류,고기요리, 관련 키워드: 무한리필
# 라벨: 오마카세, 점수: 5, 매핑 카테고리: 일식당, 관련 키워드: 오마카세
# 라벨: 마라탕, 점수: 4, 매핑 카테고리: 중식당,마라탕,양꼬치, 관련 키워드: 마라탕
# 라벨: 프랑스 코스 요리, 점수: 4, 매핑 카테고리: 프랑스음식, 관련 키워드: 코스요리,레스토랑
# 라벨: 경양식 돈까스, 점수: 3, 매핑 카테고리: 돈가스, 관련 키워드: 경양식,돈까스,돈가스
# 라벨: 한상차림식 한식백반, 점수: 2, 매핑 카테고리: 찌개,전골,한식,생선구이,기사식당, 관련 키워드: 백반,정식
# 라벨: 동네 중국집, 점수: 1, 매핑 카테고리: 중식당, 관련 키워드: 자장면,짜장면,탕수육,짬뽕
# 라벨: 분식, 점수 1, 매핑 카테고리 : 분식, 관련 키워드: 떡볶이,순대,김밥,튀김

# ### 2.3. Example Complex Cases (결정 경계 및 복합 추론 예시 - 기존 예시)
# 이 섹션은 LLM이 가장 까다로운 케이스들을 학습하는 데 사용됩니다. 각 입력 데이터 컬럼(name, category, theme_topic, theme_purpose, menu_list, review_info)이 분류에 어떻게 기여하는지 명확히 보여줍니다.

# 텍스트:
# 매장 이름: 전국구식당
# 기존 카테고리: 한식
# 테마 토픽: 가정식
# 테마 목적: 혼밥
# 메뉴 정보: 김치찌개 (얼큰한 맛), 된장찌개 (구수한 맛), 제육볶음 (매콤한 맛)
# 리뷰 정보: 집밥 같은 느낌, 반찬 푸짐해서 좋아요.

# 응답:
# ```json
# {{
# "naver_id" : "example_naver_id_1234",
# "name" : "전국구식당",
#   "대분류": "음식점",
#   "중분류": "한식",
#   "소분류": "백반,가정식",
#   "라벨": "한상차림식 한식백반",
#   "점수": "2",
#   "추론근거": "기존 category는 '한식'이지만, theme_topic '가정식', menu_list의 '김치찌개, 된장찌개', review_info '집밥 같은 느낌'을 종합할 때 '백반,가정식' 소분류에 해당하며, 특히 '한상차림식 한식백반' 라벨이 적합합니다. score_mapping에 따라 점수는 '2'입니다."
# }}
# """.strip()

# # store_crawling_20250523.json의 데이터를 동적으로 프롬프트에 추가할 함수
# def format_test_data_as_examples(test_data_list):
#     formatted_examples = ""
#     if test_data_list is None: # NoneType 에러 방지
#         return ""

#     for idx, data_item in enumerate(test_data_list):
#         naver_id = data_item.get("naver_id", "")
#         name = data_item.get("name", "")
#         category = data_item.get("category", "")

#         # 문자열 형태의 리스트/딕셔너리를 파싱하여 문자열로 변환 (None 안전하게 처리)
#         review_category = ""
#         review_category_raw = data_item.get("review_category")
#         if isinstance(review_category_raw, str):
#             try: review_category_dict = json.loads(review_category_raw.replace("'", "\""))
#             except json.JSONDecodeError: review_category_dict = {}
#             review_category = ", ".join([f"{k}: {v}" for k, v in review_category_dict.items()])
#         elif isinstance(review_category_raw, dict):
#             review_category = ", ".join([f"{k}: {v}" for k, v in review_category_raw.items()])

#         theme_mood = ""
#         theme_mood_raw = data_item.get("theme_mood")
#         if isinstance(theme_mood_raw, str):
#             try: theme_mood = ", ".join(json.loads(theme_mood_raw.replace("'", "\"")))
#             except json.JSONDecodeError: theme_mood = theme_mood_raw
#         elif isinstance(theme_mood_raw, list): theme_mood = ", ".join(theme_mood_raw)

#         theme_topic = ""
#         theme_topic_raw = data_item.get("theme_topic")
#         if isinstance(theme_topic_raw, str):
#             try: theme_topic = ", ".join(json.loads(theme_topic_raw.replace("'", "\""))) # 오타 수정 완료
#             except json.JSONDecodeError: theme_topic = theme_topic_raw
#         elif isinstance(theme_topic_raw, list): theme_topic = ", ".join(theme_topic_raw)

#         theme_purpose = ""
#         theme_purpose_raw = data_item.get("theme_purpose")
#         if isinstance(theme_purpose_raw, str):
#             try: theme_purpose = ", ".join(json.loads(theme_purpose_raw.replace("'", "\"")))
#             except json.JSONDecodeError: theme_purpose = theme_purpose_raw
#         elif isinstance(theme_purpose_raw, list): theme_purpose = ", ".join(theme_purpose_raw)


#         menu_names = [m.get("name", "") for m in data_item.get("menu_list", []) if m and m.get("name")]
#         menu_str = ", ".join(menu_names) if menu_names else "없음"

#         review_comments = [r.get("comment", "") for r in data_item.get("review_info", []) if r and r.get("comment")]
#         review_info_str = " ".join(review_comments) if review_comments else "없음"

#         formatted_examples += f"""
# ### 추가 매장 정보 예시 {idx + 1}:
# naver_id: {naver_id}
# 매장 이름: {name}
# 기존 카테고리: {category}
# 리뷰 카테고리 요약: {review_category}
# 테마 분위기: {theme_mood}
# 테마 토픽: {theme_topic}
# 테마 목적: {theme_purpose}
# 메뉴 정보: {menu_str}
# 리뷰 상세 정보: {review_info_str}
# """
#     return formatted_examples.strip()


# # 4. 관련 매핑 데이터 및 추가 예시 로드 및 문자열화
# # JSON 파일 로드 시 os.path.join을 사용하여 절대 경로를 전달
# CATEGORY_MAPPING = load_json_data(os.path.join(JSON_DATA_DIR, 'category_mapping.json'))
# SCORE_MAPPING = load_json_data(os.path.join(JSON_DATA_DIR, 'score_mapping_54321.json'))
# TEST_6_DATA = load_json_data(os.path.join(JSON_DATA_DIR, 'store_crawling_20250523.json'))

# # ---!!!! 중요 수정 부분: 파일 로드 실패 시 명확한 오류 메시지와 종료 !!!! ---
# if CATEGORY_MAPPING is None:
#     print(f"오류: '{os.path.join(JSON_DATA_DIR, 'category_mapping.json')}' 파일을 로드할 수 없습니다. 파일 경로를 확인하거나 해당 파일이 존재하는지 확인해주세요. 프로그램 종료.")
#     sys.exit(1)
# if SCORE_MAPPING is None:
#     print(f"오류: '{os.path.join(JSON_DATA_DIR, 'score_mapping_54321.json')}' 파일을 로드할 수 없습니다. 파일 경로를 확인하거나 해당 파일이 존재하는지 확인해주세요. 프로그램 종료.")
#     sys.exit(1)
# if TEST_6_DATA is None:
#     print(f"오류: '{os.path.join(JSON_DATA_DIR, 'store_crawling_20250523.json')}' 파일을 로드할 수 없습니다. 파일 경로를 확인하거나 해당 파일이 존재하는지 확인해주세요. 프로그램 종료.")
#     sys.exit(1)
# # ---!!!! 중요 수정 부분 끝 !!!! ---


# CATEGORY_MAPPING_STR = json.dumps(CATEGORY_MAPPING, ensure_ascii=False, indent=2)
# SCORE_MAPPING_STR = json.dumps(SCORE_MAPPING, ensure_ascii=False, indent=2)
# TEST_6_EXAMPLES_STR = format_test_data_as_examples(TEST_6_DATA)


# def generate_categorization_prompt(store_data: dict):
#     """
#     주어진 단일 매장 데이터를 기반으로 Gemini 모델에 보낼 프롬프트 내용을 생성합니다.
#     """
#     # 입력 데이터에서 필드를 안전하게 가져오기
#     naver_id = str(store_data.get("naver_id", ""))
#     name = store_data.get("name", "")
#     category = store_data.get("category", "")

#     review_category_raw = store_data.get("review_category", "{}")
#     review_category = ""
#     if isinstance(review_category_raw, str):
#         try: review_category_dict = json.loads(review_category_raw.replace("'", "\""))
#         except json.JSONDecodeError: review_category_dict = {}
#         review_category = ", ".join([f"{k}: {v}" for k, v in review_category_dict.items()])
#     elif isinstance(review_category_raw, dict):
#         review_category = ", ".join([f"{k}: {v}" for k, v in review_category_raw.items()])

#     theme_mood_raw = store_data.get("theme_mood")
#     theme_mood = ""
#     if isinstance(theme_mood_raw, str):
#         try: theme_mood = ", ".join(json.loads(theme_mood_raw.replace("'", "\"")))
#         except json.JSONDecodeError: theme_mood = theme_mood_raw
#     elif isinstance(theme_mood_raw, list): theme_mood = ", ".join(theme_mood_raw)

#     theme_topic_raw = store_data.get("theme_topic")
#     theme_topic = ""
#     if isinstance(theme_topic_raw, str):
#         try: theme_topic = ", ".join(json.loads(theme_topic_raw.replace("'", "\"")))
#         except json.JSONDecodeError: theme_topic = theme_topic_raw
#     elif isinstance(theme_topic_raw, list): theme_topic = ", ".join(theme_topic_raw)

#     theme_purpose_raw = store_data.get("theme_purpose")
#     theme_purpose = ""
#     if isinstance(theme_purpose_raw, str):
#         try: theme_purpose = ", ".join(json.loads(theme_purpose_raw.replace("'", "\"")))
#         except json.JSONDecodeError: theme_purpose = theme_purpose_raw
#     elif isinstance(theme_purpose_raw, list): theme_purpose = ", ".join(theme_purpose_raw)

#     menu_names = [item.get("name", "") for item in store_data.get("menu_list", []) if item and item.get("name")]
#     menu_str = ", ".join(menu_names) if menu_names else "없음"

#     review_comments = [item.get("comment", "") for item in store_data.get("review_info", []) if item and item.get("comment")]
#     review_info_str = " ".join(review_comments) if review_comments else "없음"


#     # 사용자 입력 부분을 구성
#     user_input_text = f"""
# ---
# ## 현재 분류할 매장 정보:
# naver_id: {naver_id}
# 매장 이름: {name}
# 기존 카테고리: {category}
# 리뷰 카테고리 요약: {review_category}
# 테마 분위기: {theme_mood}
# 테마 토픽: {theme_topic}
# 테마 목적: {theme_purpose}
# 메뉴 정보: {menu_str}
# 리뷰 상세 정보: {review_info_str}
# """

#     # 최종 프롬프트 구성
#     full_prompt = f"""
# {SYSTEM_PROMPT}

# {FEW_SHOT_EXAMPLES}

# ### 2.4. 추가적인 실제 매장 정보 예시 (복합 추론 학습용)
# 이 섹션은 모델이 다양한 실제 매장 데이터의 패턴을 이해하고, 이를 바탕으로 더욱 정확한 의미론적 추론을 수행하도록 돕습니다. 다음은 그 예시 데이터입니다:
# <ADDITIONAL_STORE_EXAMPLES>
# {TEST_6_EXAMPLES_STR}
# </ADDITIONAL_STORE_EXAMPLES>

# ## 관련 매핑 데이터 (참고용)

# ### 카테고리 매핑 정보:
# <CATEGORY_MAPPING_DATA>
# {CATEGORY_MAPPING_STR}
# </CATEGORY_MAPPING_DATA>

# ### 라벨별 점수 매핑 정보:
# <SCORE_MAPPING_DATA>
# {SCORE_MAPPING_STR}
# </SCORE_MAPPING_DATA>

# {user_input_text}

# ---
# 위의 지시사항, 예시, 매핑 데이터 그리고 현재 매장 정보를 바탕으로 가장 적절한 대분류, 중분류, 소분류, 라벨, 점수를 추론하고, 그 추론 근거를 포함하여 정확히 다음 JSON 스키마에 맞춰 응답하세요.
# (응답할 JSON 스키마: {json.dumps(StoreCategoryResponse.model_json_schema(), ensure_ascii=False, indent=2)})
# 응답:
# """
#     return full_prompt


# def get_categorized_store_info(store_data: dict):
#     """
#     주어진 단일 매장 정보를 Gemini 모델에 보내어 카테고리 분류 결과를 받습니다.
#     """
#     prompt = generate_categorization_prompt(store_data)

#     try:
#         # genai.GenerativeModel을 사용하여 API 호출
#         model = genai.GenerativeModel(model_name="gemini-2.0-flash")
#         response = model.generate_content(
#             contents=prompt,
#             generation_config={
#                 "response_mime_type" : "application/json",
#                 # "response_schema"
#             },
#         )

#         # 모델이 생성한 JSON 문자열 출력 (디버깅용)
#         # print(f"\n--- LLM Raw JSON Response for '{store_data.get('name', 'N/A')}' (ID: {store_data.get('naver_id', 'N/A')}) ---")
#         # print(response.text) # 이 부분에서 title 필드가 포함되었는지 확인하세요.

#         # response.parsed 대신 response.text를 직접 Pydantic으로 파싱합니다.
#         # 이렇게 하면 우리 모델의 extra='ignore' 설정이 올바르게 동작합니다.
#         validated_response = StoreCategoryResponse.model_validate_json(response.text)

#         # Pydantic 객체를 딕셔너리로 변환
#         response_dict = validated_response.model_dump()

#         # print(f"\n--- Parsed Response Object for '{store_data.get('name', 'N/A')}' ---")
#         # print(json.dumps(response_dict, indent=4, ensure_ascii=False))

#         return response_dict # 딕셔너리 반환

#     except Exception as e:
#         print(f"\n--- API 호출 또는 응답 파싱 중 오류 발생 for '{store_data.get('name', 'N/A')}' (ID: {store_data.get('naver_id', 'N/A')}) ---")
#         # 오류 발생 시 어떤 텍스트가 왔는지 확인하는 것이 디버깅에 중요합니다.
#         if 'response' in locals() and hasattr(response, 'text'):
#             print(f"--- FAILED RAW TEXT ---\n{response.text}\n-----------------------")
#         print(f"오류: {e}")
#         return None


# # --- Main 실행 블록 ---
# if __name__ == "__main__":
#     # print("--- 파일명: 'category_mapping.json', 'score_mapping_54321.json', 'store_crawling_20250523.json' ---")

#     # 파일 로드 (로컬 환경의 절대 경로를 사용)
#     # 전역 변수들이 이미 로드되어 있으므로 여기서는 추가 로드 코드를 작성하지 않습니다.

#     # store_crawling_20250523.json의 모든 매장 데이터에 대해 추론을 수행하고 결과를 모으는 로직
#     classified_results = []
#     print(f"\n--- 'store_crawling_20250523.json'에 포함된 {len(TEST_6_DATA)}개의 매장 정보 분류 시작 ---")

#     for i, store_entry in enumerate(TEST_6_DATA):
#         # print(f"\n===== {i+1}/{len(TEST_6_DATA)} 매장 분류: '{store_entry.get('name', '알 수 없음')}' =====")
#         result = get_categorized_store_info(store_entry)
#         if result:
#             classified_results.append(result) # 이미 딕셔너리이므로 그대로 추가
#         else:
#             # print(f"매장 '{store_entry.get('name', '알 수 없음')}' 분류 실패. 다음으로 넘어갑니다.")
#             # 실패한 경우에도 원본 naver_id와 name은 포함하여 기록할 수 있습니다.
#             classified_results.append({
#                 "naver_id": str(store_entry.get("naver_id", "")),
#                 "name": store_entry.get("name", ""),
#                 "대분류": "", "중분류": "", "소분류": "", "라벨": "", "점수": "",
#                 "추론근거": "분류 중 오류 발생 또는 응답 파싱 실패"
#             })
#         # print("\n" + "="*80) # 구분선 추가

#     print("\n\n--- 모든 매장 정보 분류 완료! ---")
#     print("--- 최종 분류 결과 (JSON 배열) ---")
#     print(json.dumps(classified_results, indent=4, ensure_ascii=False))

API 키가 성공적으로 검증되었습니다. Gemini API에 접근할 수 있습니다.

--- 'store_crawling_20250523.json'에 포함된 15개의 매장 정보 분류 시작 ---


KeyboardInterrupt: 

## 파이프라인 연결!
1. 이제 해야할 것은 기존 데이터 + api 입력 받고 -> Score 점수 판단해서 값 출력하도록 하는 것!
2. 끝나면 이제 네이버 자동 크롤링 한 다음에 바로 출력될 수 있도록

In [ ]:
# import json
# import os
# import google.generativeai as genai
# from pydantic import BaseModel, Field, ConfigDict
# from dotenv import load_dotenv
# from typing import Optional
# import sys
# import datetime # 날짜를 위한 모듈 추가

# # ----------------------------------------------------------------------
# # 1. 파일 경로 설정 (★★★★ 이 부분을 당신의 실제 환경에 맞게 수정해야 합니다 ★★★★)
# #    - config.env와 JSON 데이터 파일들이 위치한 디렉토리의 절대 경로를 정확히 입력하세요.
# # ----------------------------------------------------------------------

# # config.env 파일의 절대 경로
# # 예시: macOS 사용자라면 "/Users/YourUsername/Desktop/Eagle_eye_gds/config.env"
# # 예시: Windows 사용자라면 "C:\\Users\\YourUsername\\Desktop\\Eagle_eye_gds\\config.env"
# # ★★★★ 여기에 실제 config.env 파일의 절대 경로를 입력해주세요 ★★★★
# CONFIG_ENV_PATH = "/Users/10fingers/Desktop/Eagle_eye_gds/config.env" # << 이 줄을 사용자 환경에 맞게 수정

# # JSON 데이터 파일들이 위치한 디렉토리의 절대 경로 (config.env와 동일한 디렉토리로 가정)
# # ★★★★ 여기에 실제 JSON 파일들이 있는 디렉토리의 절대 경로를 입력해주세요 ★★★★
# JSON_DATA_DIR = "/Users/10fingers/Desktop/Eagle_eye_gds/Score" # << 이 줄을 사용자 환경에 맞게 수정


# # .env 파일에서 환경 변수를 로드합니다.
# load_dotenv(dotenv_path=CONFIG_ENV_PATH)

# # 환경 변수에서 API 키를 로드
# API_KEY = os.getenv("GOOGLE_API_KEY")
# if not API_KEY:
#     print(f"오류: 환경 변수 'GOOGLE_API_KEY'가 설정되지 않았습니다. '{CONFIG_ENV_PATH}' 파일을 확인해주세요.")
#     sys.exit(1)

# # Gemini 클라이언트 초기화
# genai.configure(api_key=API_KEY)

# # ----------------------------------------------------------------------
# # API 키 유효성 검증 함수 (옵션이지만 추천)
# # ----------------------------------------------------------------------
# def _check_api_key_validity():
#     """
#     설정된 Gemini API 키의 유효성을 간단한 API 호출로 검증합니다.
#     """
#     try:
#         list(genai.list_models())
#         print("API 키가 성공적으로 검증되었습니다. Gemini API에 접근할 수 있습니다.")
#         return True
#     except Exception as e:
#         print(f"오류: API 키 검증 실패. API 키가 유효하지 않거나 네트워크 문제가 있을 수 있습니다.")
#         print(f"상세 오류: {e}")
#         print("API 키를 다시 확인하거나, Google AI Studio에서 새 키를 발급받아보세요.")
#         return False

# # API 키 유효성 검증 실행
# if not _check_api_key_validity():
#     sys.exit(1)


# # 1. 매핑 및 예시 JSON 파일 로드 함수
# def load_json_data(full_file_path: str):
#     """지정된 절대 경로의 JSON 파일을 읽어 Python 객체로 반환합니다."""
#     try:
#         with open(full_file_path, 'r', encoding='utf-8') as f:
#             return json.load(f)
#     except FileNotFoundError:
#         print(f"오류: 파일 '{full_file_path}'를 찾을 수 없습니다. 파일 경로를 다시 확인해주세요.")
#         return None
#     except json.JSONDecodeError:
#         print(f"오류: 파일 '{full_file_path}'의 JSON 형식이 올바르지 않습니다. 파일 내용을 확인해주세요.")
#         return None
#     except Exception as e:
#         print(f"오류: 파일 '{full_file_path}' 로드 중 예상치 못한 오류 발생: {e}")
#         return None

# # Pydantic 모델 정의
# class StoreCategoryResponse(BaseModel):
#     model_config = ConfigDict(extra="ignore")

#     naver_id: str = Field(description="입력된 매장 정보의 고유 ID")
#     name: str = Field(description="입력된 매장 이름")
#     대분류: str = Field(description="매장 정보에서 추론된 대분류 카테고리")
#     중분류: str = Field(description="매장 정보에서 추론된 중분류 카테고리")
#     소분류: str = Field(description="매장 정보에서 추론된 소분류 카테고리 (해당 없으면 빈 문자열)")
#     # 필드명 변경: 라벨 -> 메뉴_라벨, 점수 -> 메뉴_점수, 추론근거 -> 메뉴_추론근거
#     메뉴_라벨: str = Field(description="매장 정보에서 추론된 라벨 (해당 없으면 빈 문자열)")
#     메뉴_점수: str = Field(description="추론된 라벨에 해당하는 점수 (해당 없으면 빈 문자열)")
#     메뉴_추론근거: str = Field(description="LLM이 해당 카테고리 및 라벨을 선택한 상세한 추론 과정")


# # 2. System Prompt 정의 (여기는 변경 없음)
# SYSTEM_PROMPT = """
# 당신은 대한민국 내 음식점 및 상점 카테고리 분류 전문가입니다.
# 주어진 매장 정보를 바탕으로 가장 정확한 '대분류', '중분류', '소분류', '라벨' 그리고 '점수'를 매핑해야 합니다.
# 매핑할 '라벨'이 제공된 '라벨 매핑 정보'에 존재한다면 반드시 해당 '점수'를 부여하세요.
# 만약 '라벨'이 존재하지 않아 '점수'를 부여할 수 없다면 빈 문자열('')로 표시하세요.
# 만약 해당되는 '소분류'나 '라벨'을 찾을 수 없다면 해당 필드는 빈 문자열('')로 표시하세요.
# 응답은 반드시 JSON 형식으로만 제공해야 합니다.
# 각 필드(`대분류`, `중분류`, `소분류`, `라벨`, `점수`, `추론근거`)는 문자열 타입이어야 합니다.
# '추론근거' 필드에 추론 과정을 상세히 설명해야 합니다.
# 카테고리 및 라벨 매핑 정보가 주어지지 않았다면, 주어진 텍스트 내용만을 기반으로 최선의 추론을 하되, 정보 부족을 '추론근거'에 명시하세요.
# """.strip()

# # 3. Few-Shot Examples 정의 (여기는 변경 없음 - 프롬프트 내부 라벨명은 그대로 유지)
# # LLM이 '라벨', '점수', '추론근거'라는 개념으로 학습하되, 출력 스키마는 변경된 이름으로 따르도록 지시합니다.
# FEW_SHOT_EXAMPLES = """
# ## Few-Shot Examples (예시 학습)

# ### 2.1. Example Category Mappings (카테고리 매핑 예시)
# 다음은 음식 카테고리 매핑 정보의 예시입니다:
# 대분류: 음식점, 중분류: 한식, 소분류: 한정식, 목록: ["한정식"]
# 대분류: 음식점, 중분류: 일식, 소분류: 초밥,롤, 목록: ["초밥,롤"]
# 대분류: 음식점, 중분류: 양식, 소분류: 햄버거, 목록: ["햄버거"]
# 대분류: 음식점, 중분류: 중식, 소분류: 마라탕, 목록: ["마라탕"]
# 대분류: 음식점, 중분류: 술집, 소분류: 바(BAR), 목록: ["바(BAR)", "라운지바", "루프톱바"]

# ### 2.2. Example Score Mappings (점수 매핑 예시)
# 다음은 특정 음식점 라벨에 대한 정보의 예시입니다. 각 라벨에는 점수가 부여되어 있습니다 (5가 가장 높음, 1이 가장 낮음):
# 라벨: 수제 햄버거 전문점, 점수: 5, 매핑 카테고리: 햄버거, 관련 키워드: 햄버거
# 라벨: 샤브샤브, 점수: 5, 매핑 카테고리: 샤브샤브, 관련 키워드:
# 라벨: 갈비 무한 리필, 점수: 5, 매핑 카테고리: 고기뷔페,육류,고기요리, 관련 키워드: 무한리필
# 라벨: 오마카세, 점수: 5, 매핑 카테고리: 일식당, 관련 키워드: 오마카세
# 라벨: 마라탕, 점수: 4, 매핑 카테고리: 중식당,마라탕,양꼬치, 관련 키워드: 마라탕
# 라벨: 프랑스 코스 요리, 점수: 4, 매핑 카테고리: 프랑스음식, 관련 키워드: 코스요리,레스토랑
# 라벨: 경양식 돈까스, 점수: 3, 매핑 카테고리: 돈가스, 관련 키워드: 경양식,돈까스,돈가스
# 라벨: 한상차림식 한식백반, 점수: 2, 매핑 카테고리: 찌개,전골,한식,생선구이,기사식당, 관련 키워드: 백반,정식
# 라벨: 동네 중국집, 점수: 1, 매핑 카테고리: 중식당, 관련 키워드: 자장면,짜장면,탕수육,짬뽕
# 라벨 : 분식, 점수: 1, 매핑 카테고리: 분식, 관련 키워드: 떡볶이,김밥,순대,튀김

# ### 2.3. Example Complex Cases (결정 경계 및 복합 추론 예시 - 기존 예시)
# 이 섹션은 LLM이 가장 까다로운 케이스들을 학습하는 데 사용됩니다. 각 입력 데이터 컬럼(name, category, theme_topic, theme_purpose, menu_list, review_info)이 분류에 어떻게 기여하는지 명확히 보여줍니다.

# 텍스트:
# 매장 이름: 전국구식당
# 기존 카테고리: 한식
# 테마 토픽: 가정식
# 테마 목적: 혼밥
# 메뉴 정보: 김치찌개 (얼큰한 맛), 된장찌개 (구수한 맛), 제육볶음 (매콤한 맛)
# 리뷰 정보: 집밥 같은 느낌, 반찬 푸짐해서 좋아요.

# 응답:
# ```json
# {{
# "naver_id" : "example_naver_id_1234",
# "name" : "전국구식당",
#   "대분류": "음식점",
#   "중분류": "한식",
#   "소분류": "백반,가정식",
#   "메뉴_라벨": "한상차림식 한식백반",
#   "메뉴_점수": "2",
#   "메뉴_추론근거": "기존 category는 '한식'이지만, theme_topic '가정식', menu_list의 '김치찌개, 된장찌개', review_info '집밥 같은 느낌'을 종합할 때 '백반,가정식' 소분류에 해당하며, 특히 '한상차림식 한식백반' 라벨이 적합합니다. score_mapping에 따라 점수는 '2'입니다."
# }}
# """.strip()

# # store_crawling_20250523.json의 데이터를 동적으로 프롬프트에 추가할 함수
# def format_test_data_as_examples(test_data_list):
#     formatted_examples = ""
#     if test_data_list is None:
#         return ""

#     for idx, data_item in enumerate(test_data_list):
#         naver_id = data_item.get("naver_id", "")
#         name = data_item.get("name", "")
#         category = data_item.get("category", "")

#         review_category = ""
#         review_category_raw = data_item.get("review_category")
#         if isinstance(review_category_raw, str):
#             try: review_category_dict = json.loads(review_category_raw.replace("'", "\""))
#             except json.JSONDecodeError: review_category_dict = {}
#             review_category = ", ".join([f"{k}: {v}" for k, v in review_category_dict.items()])
#         elif isinstance(review_category_raw, dict):
#             review_category = ", ".join([f"{k}: {v}" for k, v in review_category_raw.items()])

#         theme_mood = ""
#         theme_mood_raw = data_item.get("theme_mood")
#         if isinstance(theme_mood_raw, str):
#             try: theme_mood = ", ".join(json.loads(theme_mood_raw.replace("'", "\"")))
#             except json.JSONDecodeError: theme_mood = theme_mood_raw
#         elif isinstance(theme_mood_raw, list): theme_mood = ", ".join(theme_mood_raw)

#         theme_topic = ""
#         theme_topic_raw = data_item.get("theme_topic")
#         if isinstance(theme_topic_raw, str):
#             try: theme_topic = ", ".join(json.loads(theme_topic_raw.replace("'", "\"")))
#             except json.JSONDecodeError: theme_topic = theme_topic_raw
#         elif isinstance(theme_topic_raw, list): theme_topic = ", ".join(theme_topic_raw)

#         theme_purpose = ""
#         theme_purpose_raw = data_item.get("theme_purpose")
#         if isinstance(theme_purpose_raw, str):
#             try: theme_purpose = ", ".join(json.loads(theme_purpose_raw.replace("'", "\"")))
#             except json.JSONDecodeError: theme_purpose = theme_purpose_raw
#         elif isinstance(theme_purpose_raw, list): theme_purpose = ", ".join(theme_purpose_raw)

#         menu_names = [m.get("name", "") for m in data_item.get("menu_list", []) if m and m.get("name")]
#         menu_str = ", ".join(menu_names) if menu_names else "없음"

#         review_comments = [r.get("comment", "") for r in data_item.get("review_info", []) if r and r.get("comment")]
#         review_info_str = " ".join(review_comments) if review_comments else "없음"

#         formatted_examples += f"""
# ### 추가 매장 정보 예시 {idx + 1}:
# naver_id: {naver_id}
# 매장 이름: {name}
# 기존 카테고리: {category}
# 리뷰 카테고리 요약: {review_category}
# 테마 분위기: {theme_mood}
# 테마 토픽: {theme_topic}
# 테마 목적: {theme_purpose}
# 메뉴 정보: {menu_str}
# 리뷰 상세 정보: {review_info_str}
# """
#     return formatted_examples.strip()


# # 4. 관련 매핑 데이터 및 추가 예시 로드 및 문자열화
# CATEGORY_MAPPING = load_json_data(os.path.join(JSON_DATA_DIR, 'category_mapping.json'))
# SCORE_MAPPING = load_json_data(os.path.join(JSON_DATA_DIR, 'score_mapping_54321.json'))

# if CATEGORY_MAPPING is None:
#     print(f"오류: '{os.path.join(JSON_DATA_DIR, 'category_mapping.json')}' 파일을 로드할 수 없습니다. 파일 경로를 확인하거나 해당 파일이 존재하는지 확인해주세요. 프로그램 종료.")
#     sys.exit(1)
# if SCORE_MAPPING is None:
#     print(f"오류: '{os.path.join(JSON_DATA_DIR, 'score_mapping_54321.json')}' 파일을 로드할 수 없습니다. 파일 경로를 확인하거나 해당 파일이 존재하는지 확인해주세요. 프로그램 종료.")
#     sys.exit(1)


# CATEGORY_MAPPING_STR = json.dumps(CATEGORY_MAPPING, ensure_ascii=False, indent=2)
# SCORE_MAPPING_STR = json.dumps(SCORE_MAPPING, ensure_ascii=False, indent=2)


# def generate_categorization_prompt(store_data: dict, additional_examples_str: str):
#     """
#     주어진 단일 매장 데이터를 기반으로 Gemini 모델에 보낼 프롬프트 내용을 생성합니다.
#     additional_examples_str 인자를 추가하여 동적으로 추가 예시를 삽입합니다.
#     """
#     naver_id = str(store_data.get("naver_id", ""))
#     name = store_data.get("name", "")
#     category = store_data.get("category", "")

#     review_category_raw = store_data.get("review_category", "{}")
#     review_category = ""
#     if isinstance(review_category_raw, str):
#         try: review_category_dict = json.loads(review_category_raw.replace("'", "\""))
#         except json.JSONDecodeError: review_category_dict = {}
#         review_category = ", ".join([f"{k}: {v}" for k, v in review_category_dict.items()])
#     elif isinstance(review_category_raw, dict):
#         review_category = ", ".join([f"{k}: {v}" for k, v in review_category_raw.items()])

#     theme_mood_raw = store_data.get("theme_mood")
#     theme_mood = ""
#     if isinstance(theme_mood_raw, str):
#         try: theme_mood = ", ".join(json.loads(theme_mood_raw.replace("'", "\"")))
#         except json.JSONDecodeError: theme_mood = theme_mood_raw
#     elif isinstance(theme_mood_raw, list): theme_mood = ", ".join(theme_mood_raw)

#     theme_topic_raw = store_data.get("theme_topic")
#     theme_topic = ""
#     if isinstance(theme_topic_raw, str):
#         try: theme_topic = ", ".join(json.loads(theme_topic_raw.replace("'", "\"")))
#         except json.JSONDecodeError: theme_topic = theme_topic_raw
#     elif isinstance(theme_topic_raw, list): theme_topic = ", ".join(theme_topic_raw)

#     theme_purpose_raw = store_data.get("theme_purpose")
#     theme_purpose = ""
#     if isinstance(theme_purpose_raw, str):
#         try: theme_purpose = ", ".join(json.loads(theme_purpose_raw.replace("'", "\"")))
#         except json.JSONDecodeError: theme_purpose = theme_purpose_raw
#     elif isinstance(theme_purpose_raw, list): theme_purpose = ", ".join(theme_purpose_raw)

#     menu_names = [item.get("name", "") for item in store_data.get("menu_list", []) if item and item.get("name")]
#     menu_str = ", ".join(menu_names) if menu_names else "없음"

#     review_comments = [item.get("comment", "") for item in store_data.get("review_info", []) if item and item.get("comment")]
#     review_info_str = " ".join(review_comments) if review_comments else "없음"


#     # 사용자 입력 부분을 구성
#     user_input_text = f"""
# ---
# ## 현재 분류할 매장 정보:
# naver_id: {naver_id}
# 매장 이름: {name}
# 기존 카테고리: {category}
# 리뷰 카테고리 요약: {review_category}
# 테마 분위기: {theme_mood}
# 테마 토픽: {theme_topic}
# 테마 목적: {theme_purpose}
# 메뉴 정보: {menu_str}
# 리뷰 상세 정보: {review_info_str}
# """

#     # 최종 프롬프트 구성
#     full_prompt = f"""
# {SYSTEM_PROMPT}

# {FEW_SHOT_EXAMPLES}

# ### 2.4. 추가적인 실제 매장 정보 예시 (복합 추론 학습용)
# 이 섹션은 모델이 다양한 실제 매장 데이터의 패턴을 이해하고, 이를 바탕으로 더욱 정확한 의미론적 추론을 수행하도록 돕습니다. 다음은 그 예시 데이터입니다:
# <ADDITIONAL_STORE_EXAMPLES>
# {additional_examples_str}
# </ADDITIONAL_STORE_EXAMPLES>

# ## 관련 매핑 데이터 (참고용)

# ### 카테고리 매핑 정보:
# <CATEGORY_MAPPING_DATA>
# {CATEGORY_MAPPING_STR}
# </CATEGORY_MAPPING_DATA>

# ### 라벨별 점수 매핑 정보:
# <SCORE_MAPPING_DATA>
# {SCORE_MAPPING_STR}
# </SCORE_MAPPING_DATA>

# {user_input_text}

# ---
# 위의 지시사항, 예시, 매핑 데이터 그리고 현재 매장 정보를 바탕으로 가장 적절한 대분류, 중분류, 소분류, 라벨, 점수를 추론하고, 그 추론 근거를 포함하여 정확히 다음 JSON 스키마에 맞춰 응답하세요.
# (응답할 JSON 스키마: {json.dumps(StoreCategoryResponse.model_json_schema(), ensure_ascii=False, indent=2)})
# 응답:
# """
#     return full_prompt


# def get_categorized_store_info(store_data: dict, additional_examples_str: str):
#     """
#     주어진 단일 매장 정보를 Gemini 모델에 보내어 카테고리 분류 결과를 받습니다.
#     additional_examples_str 인자를 추가하여 동적으로 추가 예시를 전달합니다.
#     """
#     prompt = generate_categorization_prompt(store_data, additional_examples_str)

#     try:
#         model = genai.GenerativeModel(model_name="gemini-2.0-flash")
#         response = model.generate_content(
#             contents=prompt,
#             generation_config={
#                 "response_mime_type" : "application/json",
#             },
#         )

#         validated_response = StoreCategoryResponse.model_validate_json(response.text)
#         return validated_response.model_dump()

#     except Exception as e:
#         print(f"\n--- API 호출 또는 응답 파싱 중 오류 발생 for '{store_data.get('name', 'N/A')}' (ID: {store_data.get('naver_id', 'N/A')}) ---")
#         if 'response' in locals() and hasattr(response, 'text'):
#             print(f"--- FAILED RAW TEXT ---\n{response.text}\n-----------------------")
#         print(f"오류: {e}")
#         return None


# # --- Main 실행 블록 ---
# if __name__ == "__main__":
#     # 데이터 수집 (이 부분은 실제 크롤링 로직으로 대체되어야 합니다)
#     # 현재는 예시로 제공된 'store_crawling_20250523.json' 파일을 로드합니다.
#     # S3에서 날짜별로 파일명을 관리하고 다운로드하는 로직이 여기에 추가될 수 있습니다.
#     today_date_str = datetime.date.today().strftime("%Y%m%d")
    
#     # 테스트를 위해 고정된 파일명을 사용하거나, 실제 수집된 파일명으로 변경하세요.
#     # raw_data_filename = f"store_crawling_{today_date_str}.json"
#     raw_data_filename = "store_crawling_20250523.json" # 현재 제공된 파일명

#     # JSON_DATA_DIR은 JSON 파일들이 위치한 디렉토리입니다.
#     # full_raw_data_path = os.path.join(JSON_DATA_DIR, raw_data_filename) 
#     # 하지만 현재 테스트 환경에서 파일이 직접 제공되었으므로, 파일 경로를 직접 지정하는 것이 더 정확합니다.
#     full_raw_data_path = "store_crawling_20250523.json" 

#     raw_store_data = load_json_data(full_raw_data_path)

#     if raw_store_data is None:
#         print(f"오류: 원본 매장 데이터 파일 '{full_raw_data_path}'를 로드할 수 없습니다. 프로그램 종료.")
#         sys.exit(1)

#     # Few-shot 예시로 사용할 데이터를 포맷팅
#     # 여기서는 로드한 raw_store_data 전체를 예시로 사용할 수 있습니다.
#     # 만약 예시로 사용할 데이터가 특정 subset이라면, 해당 subset을 load_json_data로 로드하여 사용하세요.
#     TEST_EXAMPLES_FOR_PROMPT_STR = format_test_data_as_examples(raw_store_data)


#     processed_store_data = []
#     print(f"\n--- '{full_raw_data_path}'에 포함된 {len(raw_store_data)}개의 매장 정보 분류 시작 ---")

#     for i, store_entry in enumerate(raw_store_data):
#         print(f"\n===== {i+1}/{len(raw_store_data)} 매장 분류: '{store_entry.get('name', '알 수 없음')}' =====")
        
#         # 원본 데이터를 복사하여 LLM에 전달하고, 추론 결과를 여기에 추가합니다.
#         # 이렇게 하면 원본 'store_entry' 딕셔너리에 직접 수정이 가해집니다.
#         # 만약 원본 데이터를 건드리지 않고 새로운 딕셔너리를 만들고 싶다면 store_entry.copy()를 사용하세요.
        
#         # LLM 추론 결과 받기
#         llm_result = get_categorized_store_info(store_entry, TEST_EXAMPLES_FOR_PROMPT_STR)

#         # 원본 store_entry에 LLM 추론 결과 추가
#         if llm_result:
#             # Pydantic 모델에서 반환된 LLM 결과 딕셔너리에서 필요한 필드만 가져와서 원본에 추가
#             store_entry["대분류"] = llm_result.get("대분류", "")
#             store_entry["중분류"] = llm_result.get("중분류", "")
#             store_entry["소분류"] = llm_result.get("소분류", "")
#             store_entry["메뉴_라벨"] = llm_result.get("메뉴_라벨", "") # 필드명 변경 적용
#             store_entry["메뉴_점수"] = llm_result.get("메뉴_점수", "") # 필드명 변경 적용
#             store_entry["메뉴_추론근거"] = llm_result.get("메뉴_추론근거", "") # 필드명 변경 적용
#         else:
#             # LLM 추론 실패 시 빈 값으로 추가
#             store_entry["대분류"] = ""
#             store_entry["중분류"] = ""
#             store_entry["소분류"] = ""
#             store_entry["메뉴_라벨"] = "" # 필드명 변경 적용
#             store_entry["메뉴_점수"] = "" # 필드명 변경 적용
#             store_entry["메뉴_추론근거"] = "LLM 분류 중 오류 발생 또는 응답 파싱 실패"

#         processed_store_data.append(store_entry)
#         print("분류 완료.")
#         print("="*80)


#     # 최종 결과 JSON 파일로 저장
#     output_filename = f"classified_store_data_{today_date_str}.json"
#     output_filepath = os.path.join(JSON_DATA_DIR, output_filename) # JSON_DATA_DIR에 저장

#     try:
#         with open(output_filepath, 'w', encoding='utf-8') as f:
#             json.dump(processed_store_data, f, indent=4, ensure_ascii=False)
#         print(f"\n\n--- 모든 매장 정보 분류 완료! 결과가 '{output_filepath}'에 저장되었습니다. ---")
#     except Exception as e:
#         print(f"오류: 최종 JSON 파일 저장 중 오류 발생: {e}")

#     print("\n--- 최종 분류 결과 (JSON 배열 - 첫 2개 항목 예시) ---")
#     # 전체를 출력하면 너무 길어질 수 있으므로, 처음 몇 개만 출력하도록 수정
#     print(json.dumps(processed_store_data[:2], indent=4, ensure_ascii=False))
#     if len(processed_store_data) > 2:
#         print("...")

API 키가 성공적으로 검증되었습니다. Gemini API에 접근할 수 있습니다.

--- 'store_crawling_20250523.json'에 포함된 15개의 매장 정보 분류 시작 ---

===== 1/15 매장 분류: '조선김밥' =====
분류 완료.

===== 2/15 매장 분류: '스미레돈카츠 안국점' =====
분류 완료.

===== 3/15 매장 분류: '재동순두부' =====
분류 완료.

===== 4/15 매장 분류: '꽃밥에피다 북촌 친환경 그로서란트' =====
분류 완료.

===== 5/15 매장 분류: '부띠끄경성' =====
분류 완료.

===== 6/15 매장 분류: '공간' =====
분류 완료.

===== 7/15 매장 분류: '카페상국' =====
분류 완료.

===== 8/15 매장 분류: '아따블르' =====
분류 완료.

===== 9/15 매장 분류: '베이킹 도우' =====
분류 완료.

===== 10/15 매장 분류: '회화나무' =====
분류 완료.

===== 11/15 매장 분류: '커피방앗간' =====
분류 완료.

===== 12/15 매장 분류: '북촌라운지 차차티클럽' =====
분류 완료.

===== 13/15 매장 분류: '로우루프' =====
분류 완료.

===== 14/15 매장 분류: '후스테이블' =====
분류 완료.

===== 15/15 매장 분류: '베테카텐' =====
분류 완료.


--- 모든 매장 정보 분류 완료! 결과가 '/Users/10fingers/Desktop/Eagle_eye_gds/Score/classified_store_data_20250612.json'에 저장되었습니다. ---

--- 최종 분류 결과 (JSON 배열 - 첫 2개 항목 예시) ---
[
    {
        "origin_name": "조선김밥",
        "location": "종로구",
        "naver_id": 33937524,
        "n

## 위치 점수 산출 근거까지 포함하도록

In [2]:
import json
import os
import google.generativeai as genai
from pydantic import BaseModel, Field, ConfigDict
from dotenv import load_dotenv
from typing import List, Dict
import sys
import datetime
import pandas as pd
from shapely.geometry import Point, Polygon
from shapely import wkt

# ----------------------------------------------------------------------
# 1. 파일 경로 설정 (★★★★ 이 부분을 당신의 실제 환경에 맞게 수정해야 합니다 ★★★★)
#    - config.env와 JSON 데이터 파일들이 위치한 디렉토리의 절대 경로를 정확히 입력하세요.
# ----------------------------------------------------------------------

# config.env 파일의 절대 경로
CONFIG_ENV_PATH = "/Users/gidaseul/Desktop/인턴십-텐핑거스/Eagel_eye_final/.config.env"

# JSON 데이터 파일들이 위치한 디렉토리의 절대 경로
JSON_DATA_DIR = "/Users/gidaseul/Desktop/인턴십-텐핑거스/Eagel_eye_final/data"

# Polygon CSV 파일 경로 추가 (JSON_DATA_DIR과 같은 디렉토리로 가정)
HOTSPOT_POLYGON_CSV = os.path.join(JSON_DATA_DIR, "seoul_hotspots_polygons.csv")
CAMPUS_POLYGON_CSV = os.path.join(JSON_DATA_DIR, "campus_polygons.csv")


# .env 파일에서 환경 변수를 로드합니다.
load_dotenv(dotenv_path=CONFIG_ENV_PATH)

# 환경 변수에서 API 키를 로드
API_KEY = os.getenv("GOOGLE_API_KEY")
if not API_KEY:
    print(f"오류: 환경 변수 'GOOGLE_API_KEY'가 설정되지 않았습니다. '{CONFIG_ENV_PATH}' 파일을 확인해주세요.")
    sys.exit(1)

# Gemini 클라이언트 초기화
genai.configure(api_key=API_KEY)

# ----------------------------------------------------------------------
# API 키 유효성 검증 함수 (옵션이지만 추천)
# ----------------------------------------------------------------------
def _check_api_key_validity():
    """
    설정된 Gemini API 키의 유효성을 간단한 API 호출로 검증합니다.
    """
    try:
        list(genai.list_models())
        print("API 키가 성공적으로 검증되었습니다. Gemini API에 접근할 수 있습니다.")
        return True
    except Exception as e:
        print(f"오류: API 키 검증 실패. API 키가 유효하지 않거나 네트워크 문제가 있을 수 있습니다.")
        print(f"상세 오류: {e}")
        print("API 키를 다시 확인하거나, Google AI Studio에서 새 키를 발급받아보세요.")
        return False

# API 키 유효성 검증 실행
if not _check_api_key_validity():
    sys.exit(1)


# 1. 매핑 및 예시 JSON 파일 로드 함수
def load_json_data(full_file_path: str):
    """지정된 절대 경로의 JSON 파일을 읽어 Python 객체로 반환합니다."""
    try:
        with open(full_file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"오류: 파일 '{full_file_path}'를 찾을 수 없습니다. 파일 경로를 다시 확인해주세요.")
        return None
    except json.JSONDecodeError:
        print(f"오류: 파일 '{full_file_path}'의 JSON 형식이 올바르지 않습니다. 파일 내용을 확인해주세요.")
        return None
    except Exception as e:
        print(f"오류: 파일 '{full_file_path}' 로드 중 예상치 못한 오류 발생: {e}")
        return None

# Pydantic 모델 정의
class StoreCategoryResponse(BaseModel):
    model_config = ConfigDict(extra="ignore")

    naver_id: str = Field(description="입력된 매장 정보의 고유 ID")
    name: str = Field(description="입력된 매장 이름")
    대분류: str = Field(description="매장 정보에서 추론된 대분류 카테고리")
    중분류: str = Field(description="매장 정보에서 추론된 중분류 카테고리")
    소분류: str = Field(description="매장 정보에서 추론된 소분류 카테고리 (해당 없으면 빈 문자열)")
    메뉴_라벨: str = Field(description="매장 정보에서 추론된 라벨 (해당 없으면 빈 문자열)")
    메뉴_점수: str = Field(description="추론된 라벨에 해당하는 점수 (해당 없으면 빈 문자열)")
    메뉴_추론근거: str = Field(description="LLM이 해당 카테고리 및 라벨을 선택한 상세한 추론 과정")


# 2. System Prompt 정의
SYSTEM_PROMPT = """
당신은 대한민국 내 음식점 및 상점 카테고리 분류 전문가입니다.
주어진 매장 정보를 바탕으로 가장 정확한 '대분류', '중분류', '소분류', '라벨' 그리고 '점수'를 매핑해야 합니다.
매핑할 '라벨'이 제공된 '라벨 매핑 정보'에 존재한다면 반드시 해당 '점수'를 부여하세요.
**만약 '라벨'이 존재하지 않아 '점수'를 부여할 수 없다면, `메뉴_라벨`과 `메뉴_점수` 필드를 반드시 빈 문자열('')로 표시해야 합니다.**
**마찬가지로, 해당되는 '소분류'를 찾을 수 없다면 `소분류` 필드를 빈 문자열('')로 표시해야 합니다.**
응답은 반드시 JSON 형식으로만 제공해야 합니다.
각 필드(`대분류`, `중분류`, `소분류`, `메뉴_라벨`, `메뉴_점수`, `메뉴_추론근거`)는 문자열 타입이어야 합니다.
'메뉴_추론근거' 필드에 추론 과정을 상세히 설명해야 합니다. **라벨이 없는 경우에도 '메뉴_추론근거'는 반드시 포함되어야 하며, 라벨이 없는 이유를 설명하세요.**
카테고리 및 라벨 매핑 정보가 주어지지 않았다면, 주어진 텍스트 내용만을 기반으로 최선의 추론을 하되, 정보 부족을 '메뉴_추론근거'에 명시하세요.
""".strip()

# 3. Few-Shot Examples 정의
FEW_SHOT_EXAMPLES = """
## Few-Shot Examples (예시 학습)

### 2.1. Example Category Mappings (카테고리 매핑 예시)
다음은 음식 카테고리 매핑 정보의 예시입니다:
대분류: 음식점, 중분류: 한식, 소분류: 한정식, 목록: ["한정식"]
대분류: 음식점, 중분류: 일식, 소분류: 초밥,롤, 목록: ["초밥,롤"]
대분류: 음식점, 중분류: 양식, 소분류: 햄버거, 목록: ["햄버거"]
대분류: 음식점, 중분류: 중식, 소분류: 마라탕, 목록: ["마라탕"]
대분류: 음식점, 중분류: 술집, 소분류: 바(BAR), 목록: ["바(BAR)", "라운지바", "루프톱바"]

### 2.2. Example Score Mappings (점수 매핑 예시)
다음은 특정 음식점 라벨에 대한 정보의 예시입니다. 각 라벨에는 점수가 부여되어 있습니다 (5가 가장 높음, 1이 가장 낮음):
라벨: 수제 햄버거 전문점, 점수: 5, 매핑 카테고리: 햄버거, 관련 키워드: 햄버거
라벨: 샤브샤브, 점수: 5, 매핑 카테고리: 샤브샤브, 관련 키워드:
라벨: 갈비 무한 리필, 점수: 5, 매핑 카테고리: 고기뷔페,육류,고기요리, 관련 키워드: 무한리필
라벨: 오마카세, 점수: 5, 매핑 카테고리: 일식당, 관련 키워드: 오마카세
라벨: 마라탕, 점수: 4, 매핑 카테고리: 중식당,마라탕,양꼬치, 관련 키워드: 마라탕
라벨: 프랑스 코스 요리, 점수: 4, 매핑 카테고리: 프랑스음식, 관련 키워드: 코스요리,레스토랑
라벨: 경양식 돈까스, 점수: 3, 매핑 카테고리: 돈가스, 관련 키워드: 경양식,돈까스,돈가스
라벨: 한상차림식 한식백반, 점수: 2, 매핑 카테고리: 찌개,전골,한식,생선구이,기사식당, 관련 키워드: 백반,정식
라벨: 동네 중국집, 점수: 1, 매핑 카테고리: 중식당, 관련 키워드: 자장면,짜장면,탕수육,짬뽕
라벨 : 분식, 점수: 1, 매핑 카테고리: 분식, 관련 키워드: 떡볶이,김밥,순대,튀김

### 2.3. Example Complex Cases (결정 경계 및 복합 추론 예시 - 기존 예시)
이 섹션은 LLM이 가장 까다로운 케이스들을 학습하는 데 사용됩니다. 각 입력 데이터 컬럼(name, category, theme_topic, theme_purpose, menu_list, review_info)이 분류에 어떻게 기여하는지 명확히 보여줍니다.

### 라벨이 있는 경우

텍스트:
매장 이름: 전국구식당
기존 카테고리: 한식
테마 토픽: 가정식
테마 목적: 혼밥
메뉴 정보: 김치찌개 (얼큰한 맛), 된장찌개 (구수한 맛), 제육볶음 (매콤한 맛)
리뷰 정보: 집밥 같은 느낌, 반찬 푸짐해서 좋아요.

응답:
```json
{{
"naver_id" : "example_naver_id_1234",
"name" : "전국구식당",
  "대분류": "음식점",
  "중분류": "한식",
  "소분류": "백반,가정식",
  "라벨": "한상차림식 한식백반",
  "점수": "2",
  "추론근거": "기존 category는 '한식'이지만, theme_topic '가정식', menu_list의 '김치찌개, 된장찌개', review_info '집밥 같은 느낌'을 종합할 때 '백반,가정식' 소분류에 해당하며, 특히 '한상차림식 한식백반' 라벨이 적합합니다. score_mapping에 따라 점수는 '2'입니다."
}}

### 라벨이 없는 경우

텍스트:
매장 이름: 예쁜카페
기존 카테고리: 카페
테마 토픽: 뷰맛집
테마 목적: 데이트
메뉴 정보: 아메리카노, 카페라떼, 티라미수 케이크
리뷰 상세 정보: 창덕궁 뷰가 너무 예뻐요, 디저트도 맛있어요.

응답:
```json
{{}
  "naver_id" : "example_naver_id_cafe",
  "name" : "예쁜카페",
  "대분류": "음식점",
  "중분류": "카페",
  "소분류": "",
  "메뉴_라벨": "",
  "메뉴_점수": "",
  "메뉴_추론근거": "기존 카테고리가 '카페'이고, 메뉴와 리뷰에서 커피 및 디저트 관련 정보가 확인됩니다. 제공된 라벨 매핑 정보에는 해당되는 특정 카페 라벨이 없어 빈 문자열로 처리했습니다."
}}
""".strip()

# store_crawling_20250523.json의 데이터를 동적으로 프롬프트에 추가할 함수
def format_test_data_as_examples(test_data_list):
    formatted_examples = ""
    if test_data_list is None:
        return ""

    for idx, data_item in enumerate(test_data_list):
        naver_id = data_item.get("naver_id", "")
        name = data_item.get("name", "")
        category = data_item.get("category", "")

        review_category = ""
        review_category_raw = data_item.get("review_category")
        if isinstance(review_category_raw, str):
            try: review_category_dict = json.loads(review_category_raw.replace("'", "\""))
            except json.JSONDecodeError: review_category_dict = {}
            review_category = ", ".join([f"{k}: {v}" for k, v in review_category_dict.items()])
        elif isinstance(review_category_raw, dict):
            review_category = ", ".join([f"{k}: {v}" for k, v in review_category_raw.items()])

        theme_mood = ""
        theme_mood_raw = data_item.get("theme_mood")
        if isinstance(theme_mood_raw, str):
            try: theme_mood = ", ".join(json.loads(theme_mood_raw.replace("'", "\"")))
            except json.JSONDecodeError: theme_mood = theme_mood_raw
        elif isinstance(theme_mood_raw, list): theme_mood = ", ".join(theme_mood_raw)

        theme_topic = ""
        theme_topic_raw = data_item.get("theme_topic")
        if isinstance(theme_topic_raw, str):
            try: theme_topic = ", ".join(json.loads(theme_topic_raw.replace("'", "\"")))
            except json.JSONDecodeError: theme_topic = theme_topic_raw
        elif isinstance(theme_topic_raw, list): theme_topic = ", ".join(theme_topic_raw)

        theme_purpose = ""
        theme_purpose_raw = data_item.get("theme_purpose")
        if isinstance(theme_purpose_raw, str):
            try: theme_purpose = ", ".join(json.loads(theme_purpose_raw.replace("'", "\"")))
            except json.JSONDecodeError: theme_purpose = theme_purpose_raw
        elif isinstance(theme_purpose_raw, list): theme_purpose = ", ".join(theme_purpose_raw)

        menu_names = [m.get("name", "") for m in data_item.get("menu_list", []) if m and m.get("name")]
        menu_str = ", ".join(menu_names) if menu_names else "없음"

        review_comments = [r.get("comment", "") for r in data_item.get("review_info", []) if r and r.get("comment")]
        review_info_str = " ".join(review_comments) if review_comments else "없음"

        


        formatted_examples += f"""
### 추가 매장 정보 예시 {idx + 1}:
naver_id: {naver_id}
매장 이름: {name}
기존 카테고리: {category}
리뷰 카테고리 요약: {review_category}
테마 분위기: {theme_mood}
테마 토픽: {theme_topic}
테마 목적: {theme_purpose}
메뉴 정보: {menu_str}
리뷰 상세 정보: {review_info_str}
"""
    return formatted_examples.strip()


# 4. 관련 매핑 데이터 및 추가 예시 로드 및 문자열화
CATEGORY_MAPPING = load_json_data(os.path.join(JSON_DATA_DIR, 'category_mapping.json'))
SCORE_MAPPING = load_json_data(os.path.join(JSON_DATA_DIR, 'score_mapping_54321.json'))

if CATEGORY_MAPPING is None:
    print(f"오류: '{os.path.join(JSON_DATA_DIR, 'category_mapping.json')}' 파일을 로드할 수 없습니다. 파일 경로를 확인하거나 해당 파일이 존재하는지 확인해주세요.")
    sys.exit(1)
if SCORE_MAPPING is None:
    print(f"오류: '{os.path.join(JSON_DATA_DIR, 'score_mapping_54321.json')}' 파일을 로드할 수 없습니다. 파일 경로를 확인하거나 해당 파일이 존재하는지 확인해주세요.")
    sys.exit(1)


CATEGORY_MAPPING_STR = json.dumps(CATEGORY_MAPPING, ensure_ascii=False, indent=2)
SCORE_MAPPING_STR = json.dumps(SCORE_MAPPING, ensure_ascii=False, indent=2)


def generate_categorization_prompt(store_data: dict, additional_examples_str: str):
    """
    주어진 단일 매장 데이터를 기반으로 Gemini 모델에 보낼 프롬프트 내용을 생성합니다.
    additional_examples_str 인자를 추가하여 동적으로 추가 예시를 삽입합니다.
    """
    naver_id = str(store_data.get("naver_id", ""))
    name = store_data.get("name", "")
    category = store_data.get("category", "")

    review_category_raw = store_data.get("review_category", "{}")
    review_category = ""
    if isinstance(review_category_raw, str):
        try: review_category_dict = json.loads(review_category_raw.replace("'", "\""))
        except json.JSONDecodeError: review_category_dict = {}
        review_category = ", ".join([f"{k}: {v}" for k, v in review_category_dict.items()])
    elif isinstance(review_category_raw, dict):
        review_category = ", ".join([f"{k}: {v}" for k, v in review_category_raw.items()])

    theme_mood_raw = store_data.get("theme_mood")
    theme_mood = ""
    if isinstance(theme_mood_raw, str):
        try: theme_mood = ", ".join(json.loads(theme_mood_raw.replace("'", "\"")))
        except json.JSONDecodeError: theme_mood = theme_mood_raw
    elif isinstance(theme_mood_raw, list): theme_mood = ", ".join(theme_mood_raw)

    theme_topic_raw = store_data.get("theme_topic")
    theme_topic = ""
    if isinstance(theme_topic_raw, str):
        try: theme_topic = ", ".join(json.loads(theme_topic_raw.replace("'", "\"")))
        except json.JSONDecodeError: theme_topic = theme_topic_raw
    elif isinstance(theme_topic_raw, list): theme_topic = ", ".join(theme_topic_raw)

    theme_purpose_raw = store_data.get("theme_purpose")
    theme_purpose = ""
    if isinstance(theme_purpose_raw, str):
        try: theme_purpose = ", ".join(json.loads(theme_purpose_raw.replace("'", "\"")))
        except json.JSONDecodeError: theme_purpose = theme_purpose_raw
    elif isinstance(theme_purpose_raw, list): theme_purpose = ", ".join(theme_purpose_raw)

    menu_names = [item.get("name", "") for item in store_data.get("menu_list", []) if item and item.get("name")]
    menu_str = ", ".join(menu_names) if menu_names else "없음"

    review_comments = [item.get("comment", "") for item in store_data.get("review_info", []) if item and item.get("comment")]
    review_info_str = " ".join(review_comments) if review_comments else "없음"


    # 사용자 입력 부분을 구성
    user_input_text = f"""
---
## 현재 분류할 매장 정보:
naver_id: {naver_id}
매장 이름: {name}
기존 카테고리: {category}
리뷰 카테고리 요약: {review_category}
테마 분위기: {theme_mood}
테마 토픽: {theme_topic}
테마 목적: {theme_purpose}
메뉴 정보: {menu_str}
리뷰 상세 정보: {review_info_str}
"""

    # 최종 프롬프트 구성
    full_prompt = f"""
{SYSTEM_PROMPT}

{FEW_SHOT_EXAMPLES}

### 2.4. 추가적인 실제 매장 정보 예시 (복합 추론 학습용)
이 섹션은 모델이 다양한 실제 매장 데이터의 패턴을 이해하고, 이를 바탕으로 더욱 정확한 의미론적 추론을 수행하도록 돕습니다. 다음은 그 예시 데이터입니다:
<ADDITIONAL_STORE_EXAMPLES>
{additional_examples_str}
</ADDITIONAL_STORE_EXAMPLES>

## 관련 매핑 데이터 (참고용)

### 카테고리 매핑 정보:
<CATEGORY_MAPPING_DATA>
{CATEGORY_MAPPING_STR}
</CATEGORY_MAPPING_DATA>

### 라벨별 점수 매핑 정보:
<SCORE_MAPPING_DATA>
{SCORE_MAPPING_STR}
</SCORE_MAPPING_DATA>

{user_input_text}

---
위의 지시사항, 예시, 매핑 데이터 그리고 현재 매장 정보를 바탕으로 가장 적절한 대분류, 중분류, 소분류, 라벨, 점수를 추론하고, 그 추론 근거를 포함하여 정확히 다음 JSON 스키마에 맞춰 응답하세요.
(응답할 JSON 스키마: {json.dumps(StoreCategoryResponse.model_json_schema(), ensure_ascii=False, indent=2)})
응답:
"""
    return full_prompt


def get_categorized_store_info(store_data: dict, additional_examples_str: str):
    """
    주어진 단일 매장 정보를 Gemini 모델에 보내어 카테고리 분류 결과를 받습니다.
    additional_examples_str 인자를 추가하여 동적으로 추가 예시를 전달합니다.
    """
    prompt = generate_categorization_prompt(store_data, additional_examples_str)

    try:
        model = genai.GenerativeModel(model_name="gemini-2.0-flash")
        response = model.generate_content(
            contents=prompt,
            generation_config={
                "response_mime_type" : "application/json",
            },
            request_options={"timeout": 15} # 15초 타임아웃 추가

        )
        print(f"DEBUG: Raw LLM response text:\n{response.text}") # 이 부분을 추가
        validated_response = StoreCategoryResponse.model_validate_json(response.text)
        return validated_response.model_dump()

    except Exception as e:
        print(f"\n--- API 호출 또는 응답 파싱 중 오류 발생 for '{store_data.get('name', 'N/A')}' (ID: {store_data.get('naver_id', 'N/A')}) ---")
        if 'response' in locals() and hasattr(response, 'text'):
            print(f"--- FAILED RAW TEXT ---\n{response.text}\n-----------------------")
        print(f"오류: {e}")
        return None

# 🔹 polygon dict 로딩 함수 (사용자 제공 코드 기반)
def load_polygons_from_df(file_path: str, name_col: str, polygon_col: str) -> Dict[str, Polygon]:
    polygons = {}
    try:
        df = pd.read_csv(file_path)
        if name_col not in df.columns or polygon_col not in df.columns:
            print(f"경고: '{file_path}'에 '{name_col}' 또는 '{polygon_col}' 컬럼이 없습니다.")
            return {}

        for _, row in df.iterrows():
            try:
                name = row[name_col]
                # wkt.loads는 문자열 인코딩 문제에 덜 민감
                poly = wkt.loads(row[polygon_col])
                polygons[name] = poly
            except Exception as e:
                # print(f"경고: '{file_path}'에서 Polygon 파싱 실패 (이름: {row.get(name_col, 'N/A')}): {e}")
                continue # 실패한 폴리곤은 건너뛰기
    except FileNotFoundError:
        print(f"오류: Polygon 파일 '{file_path}'를 찾을 수 없습니다. 경로를 확인해주세요.")
    except Exception as e:
        print(f"오류: Polygon 파일 '{file_path}' 로드 중 예상치 못한 오류 발생: {e}")
    return polygons

# 🔹 점수 계산 함수 (사용자 제공 코드 기반, 반환값 필드명 및 실패사유 조정)
def calculate_location_score(
    row: dict,
    hotplace_polys: Dict[str, Polygon],
    campus_polys: Dict[str, Polygon],
    new_hot_keywords: List[str]
) -> dict:
    lat, lng = row.get("gps_latitude"), row.get("gps_longitude")
    address = str(row.get("address", ""))
    distance = pd.to_numeric(row.get("distance_from_subway", None), errors="coerce")

    # 1. 위경도 누락 체크
    if pd.isna(lat) or pd.isna(lng):
        return {
            "위치_점수": 0.0,
            "위치_산출근거": "",
            "위치_실패사유": "gps 데이터 없음"
        }

    point = Point(lng, lat)

    # 2. 핫플레이스 Polygon (5점)
    for name, poly in hotplace_polys.items():
        if point.within(poly):
            return {
                "위치_점수": 5.0,
                "위치_산출근거": "핫플레이스",
                "위치_실패사유": "정상적으로 작동함"
            }

    # 3. 신규 핫플레이스 키워드 (4점) - 주소 기반 (Polygon 아님)
    for keyword in new_hot_keywords:
        if keyword in address:
            return {
                "위치_점수": 4.0,
                "위치_산출근거": "신규_핫플레이스",
                "위치_실패사유": "정상적으로 작동함"
            }

    # 4. 대학가 Polygon (4점)
    for name, poly in campus_polys.items():
        if point.within(poly):
            return {
                "위치_점수": 4.0,
                "위치_산출근거": "대학가",
                "위치_실패사유": "정상적으로 작동함"
            }

    # 5. 지하철 거리 기반 점수 (3점) - 이전 조건에 모두 미매칭 시
    if pd.isna(distance):
        return {
            "위치_점수": 0.0,
            "위치_산출근거": "",
            "위치_실패사유": "지하철역 거리 데이터 없음"
        }

    if distance <= 900: # 도보 15분 이내 (900m)
        return {
            "위치_점수": 3.0,
            "위치_산출근거": "거리_도보15분",
            "위치_실패사유": "정상적으로 작동함"
        }
    
    # 6. 모든 조건에서 미매칭 (2점 또는 1점 기준이 명확하지 않아 '모든 매칭이 안되었음'으로 처리)
    # 현재 사용자 제공 로직에서는 900m 초과 시 바로 unmatched로 넘어감.
    # 2점 ('역에서 도보 25분 이내 (1000m 이상)') 또는 1점 ('역에서 버스 환승 필요') 기준을 추가하려면
    # 이 부분에 elif distance > 900: (2점 로직) 및 else: (1점 로직)을 추가해야 합니다.
    # 현재는 900m 초과하는 모든 거리는 '모든 매칭이 안되었음'으로 간주됩니다.
    return {
        "위치_점수": 0.0, # 2점 또는 1점으로 변경될 수 있음
        "위치_산출근거": "거리_도보25분" if distance > 900 else "", # 900m 초과 시 거리_도보25분으로 표기
        "위치_실패사유": "모든 매칭이 안되었음"
    }


# --- Main 실행 블록 ---
if __name__ == "__main__":
    today_date_str = datetime.date.today().strftime("%Y%m%d")
    raw_data_filename = "가디.json" # 입력값으로 들어올 데이터

    # JSON_DATA_DIR을 사용하여 파일을 찾도록 변경
    full_raw_data_path = os.path.join(JSON_DATA_DIR, raw_data_filename) 
    
    # 로컬 테스트를 위해 현재 디렉토리에도 파일이 있는지 확인
    if not os.path.exists(full_raw_data_path):
        if os.path.exists(raw_data_filename):
            full_raw_data_path = raw_data_filename
        else:
            print(f"오류: 원본 매장 데이터 파일 '{raw_data_filename}'을 지정된 경로 '{JSON_DATA_DIR}'에서도, 현재 디렉토리에서도 찾을 수 없습니다. 경로를 확인해주세요.")
            sys.exit(1)


    raw_store_data = load_json_data(full_raw_data_path)

    if raw_store_data is None:
        print(f"오류: 원본 매장 데이터 파일 '{full_raw_data_path}'를 로드할 수 없습니다. 프로그램 종료.")
        sys.exit(1)

    # Few-shot 예시로 사용할 데이터를 포맷팅
    TEST_EXAMPLES_FOR_PROMPT_STR = format_test_data_as_examples(raw_store_data)

    # ★★★ Polygon 데이터 로드 ★★★
    print(f"\n--- 핫플레이스 Polygon 데이터 로드 중: {HOTSPOT_POLYGON_CSV} ---")
    hotplace_polys = load_polygons_from_df(HOTSPOT_POLYGON_CSV, "location", "polygon_str")
    print(f"로딩된 핫플레이스 Polygon 개수: {len(hotplace_polys)}")

    print(f"\n--- 대학가 Polygon 데이터 로드 중: {CAMPUS_POLYGON_CSV} ---")
    campus_polys = load_polygons_from_df(CAMPUS_POLYGON_CSV, "campus_name", "polygon_str")
    print(f"로딩된 대학가 Polygon 개수: {len(campus_polys)}")
    
    # 로드 실패 시 경고 메시지는 이미 load_polygons_from_df에서 출력되므로,
    # 여기서는 폴리곤 데이터가 비어있을 경우에 대한 추가적인 에러 처리 (예: 프로그램 종료)를 결정할 수 있습니다.
    # 현재는 데이터가 없어도 계속 진행되도록 둡니다.

    # 신규 핫플레이스 키워드 정의 (사용자 제공 코드에서 가져옴)
    new_hot_keywords = ["삼성역", "코엑스", "익선동", "샤로수길", "송리단길", "해방촌", "후암동", "서촌"]


    processed_store_data = []
    print(f"\n--- '{full_raw_data_path}'에 포함된 {len(raw_store_data)}개의 매장 정보 분류 시작 ---")

    for i, store_entry in enumerate(raw_store_data):
        print(f"\n===== {i+1}/{len(raw_store_data)} 매장 분류: '{store_entry.get('name', '알 수 없음')}' =====")
        
        # 1. LLM 추론 결과 받기 (메뉴 관련 점수)
        llm_result = get_categorized_store_info(store_entry, TEST_EXAMPLES_FOR_PROMPT_STR)

        # 2. 위치 점수 계산 (로드된 Polygon 데이터와 키워드를 전달)
        location_score_info = calculate_location_score(store_entry, hotplace_polys, campus_polys, new_hot_keywords)

        # 원본 store_entry에 LLM 추론 결과와 위치 점수 결과 추가
        # 메뉴 관련 필드 추가
        if llm_result:
            store_entry["대분류"] = llm_result.get("대분류", "")
            store_entry["중분류"] = llm_result.get("중분류", "")
            store_entry["소분류"] = llm_result.get("소분류", "")
            store_entry["메뉴_라벨"] = llm_result.get("메뉴_라벨", "")
            # 메뉴_점수 변환 로직 (LLM이 빈 문자열 줄 수 있으므로)
            try:
                # LLM이 반환한 '메뉴_점수'를 float으로 변환 시도
                # 만약 LLM이 빈 문자열 ''을 반환하면 ValueError 발생
                menu_score_from_llm = float(llm_result.get("메뉴_점수", 0)) # 0으로 get하면 int 타입이므로 float(0)
            except (ValueError, TypeError): # 빈 문자열이나 None 등 변환 불가 시
                menu_score_from_llm = 0.0 # 기본값으로 0.0 할당
            
            store_entry["메뉴_점수"] = menu_score_from_llm
            store_entry["메뉴_추론근거"] = llm_result.get("메뉴_추론근거", "")
        else:
            store_entry["대분류"] = ""
            store_entry["중분류"] = ""
            store_entry["소분류"] = ""
            store_entry["메뉴_라벨"] = ""
            store_entry["메뉴_점수"] = 0.0
            store_entry["메뉴_추론근거"] = "LLM 분류 중 오류 발생 또는 응답 파싱 실패"
        
        # 위치 관련 필드 추가
        store_entry["위치_점수"] = location_score_info.get("위치_점수", 0.0)
        store_entry["위치_산출근거"] = location_score_info.get("위치_산출근거", "")
        store_entry["위치_실패사유"] = location_score_info.get("위치_실패사유", "")

        # 3. Total 점수 합산 (메뉴 점수, 위치 점수, 추가 조건 점수)
        base_total_score = (store_entry["메뉴_점수"] + store_entry.get("위치_점수", 0.0)) /2 # 위치 점수는 2로 나누어 평균을 구함
        additional_score = 0.0
        total_score_breakdown = []

        # on_tv
        if store_entry.get("on_tv") == True:
            additional_score += 0.3
            total_score_breakdown.append("방송 출연 (+0.3점)")

        # seoul_michelin
        if store_entry.get("seoul_michelin") == True:
            additional_score += 0.5
            total_score_breakdown.append("서울 미쉐린 선정 (+0.5점)")

        # blog_review_count
        blog_review_count = store_entry.get("blog_review_count")
        if isinstance(blog_review_count, (int, float)) and blog_review_count >= 300:
            additional_score += 0.3
            total_score_breakdown.append(f"블로그 리뷰 300개 이상 ({int(blog_review_count)}개) (+0.3점)")

        # parking_available
        if store_entry.get("parking_available") == True:
            additional_score += 0.2
            total_score_breakdown.append("주차 가능 (+0.2점)")
        
        # 여기서는 "Total_점수 = [base_total_score는 ((메뉴점수 + 위치점수) / 2 )]+ [additional_score는 추가점수]" 로 구현합니다.

        final_total_score = base_total_score  + additional_score
        store_entry["Total_점수"] = final_total_score


        # Total_산출근거 구성
        total_score_reason_parts = [
            f"메뉴 점수({store_entry['메뉴_점수']:.1f}점) + 위치 점수({store_entry['위치_점수']:.1f}점) / 2 = {base_total_score:.1f}점"
        ]
        # 추가 점수 항목이 있을 경우 상세 내역 추가
        if total_score_breakdown:
            # 각 항목을 "항목명(점수)" 형식으로 변환
            formatted_additional_items = []
            for item_str in total_score_breakdown:
                # "방송 출연 (+0.3점)" -> "방송 출연(0.3점)"
                # "블로그 리뷰 300개 이상 (300개) (+0.3점)" -> "블로그 리뷰 300개 이상(0.3점)"
                parts = item_str.split('(')
                name = parts[0].strip()
                score_part = parts[-1].replace('점)', '').replace('+', '').strip()
                try:
                    score = float(score_part)
                    if "개)" in item_str: # 블로그 리뷰처럼 개수 정보가 있는 경우
                         count_part = item_str.split('(')[1].split(')')[0]
                         formatted_additional_items.append(f"{name} ({count_part}) ({score:.1f}점)")
                    else:
                         formatted_additional_items.append(f"{name} ({score:.1f}점)")
                except ValueError:
                    formatted_additional_items.append(name) # 점수 파싱 실패 시 이름만
            
            total_score_reason_parts.append(f"추가 점수 항목: {', '.join(formatted_additional_items)}")
            total_score_reason_parts.append(f"총 추가 점수: {additional_score:.1f}점")
        
        store_entry["Total_산출근거"] = "; ".join(total_score_reason_parts)

        processed_store_data.append(store_entry)
        print("분류 및 점수 계산 완료.")
        print("="*80)



    # 최종 결과 JSON 파일로 저장
    output_filename = f"classified_store_data_{today_date_str}.json"
    output_filepath = os.path.join(JSON_DATA_DIR, output_filename)

    try:
        with open(output_filepath, 'w', encoding='utf-8') as f:
            json.dump(processed_store_data, f, indent=4, ensure_ascii=False)
        print(f"\n\n--- 모든 매장 정보 분류 완료! 결과가 '{output_filepath}'에 저장되었습니다. ---")
    except Exception as e:
        print(f"오류: 최종 JSON 파일 저장 중 오류 발생: {e}")

    print("\n--- 최종 분류 결과 (JSON 배열 - 첫 2개 항목 예시) ---")
    print(json.dumps(processed_store_data[:2], indent=4, ensure_ascii=False))
    if len(processed_store_data) > 2:
        print("...")

API 키가 성공적으로 검증되었습니다. Gemini API에 접근할 수 있습니다.

--- 핫플레이스 Polygon 데이터 로드 중: /Users/gidaseul/Desktop/인턴십-텐핑거스/Eagel_eye_final/data/seoul_hotspots_polygons.csv ---
로딩된 핫플레이스 Polygon 개수: 14

--- 대학가 Polygon 데이터 로드 중: /Users/gidaseul/Desktop/인턴십-텐핑거스/Eagel_eye_final/data/campus_polygons.csv ---
로딩된 대학가 Polygon 개수: 10

--- '/Users/gidaseul/Desktop/인턴십-텐핑거스/Eagel_eye_final/data/가디.json'에 포함된 2개의 매장 정보 분류 시작 ---

===== 1/2 매장 분류: '하남돼지집 가산디지털역점' =====
DEBUG: Raw LLM response text:
{
  "naver_id": "37815226",
  "name": "하남돼지집 가산디지털역점",
  "대분류": "음식점",
  "중분류": "한식",
  "소분류": "육류,고기요리",
  "메뉴_라벨": "돼지고기 구이",
  "메뉴_점수": "4",
  "메뉴_추론근거": "기존 카테고리가 '돼지고기구이'이고, 리뷰 카테고리 요약에서 '음식이 맛있어요', '고기 질이 좋아요' 등의 긍정적인 평가가 많습니다. 테마 토픽에는 '생삼겹살'이 언급되어 있으며, 메뉴 정보에도 삼겹살, 목살, 갈비 등 돼지고기 메뉴가 다양하게 있습니다. 따라서 '육류,고기요리' 소분류에 해당하며, '돼지고기 구이' 라벨이 적합합니다. score_mapping에 따라 점수는 '4'입니다."
}
분류 및 점수 계산 완료.

===== 2/2 매장 분류: '김가네곱창구이 대림역본점' =====
DEBUG: Raw LLM response text:
{
  "naver_id": "13438729",
  "name": "김가네곱창구이 대림역본점",
 

In [ ]:
- 전체 점수에서 total 점수만 추가하고
- 크롤링 <-> 점수 산출 (파이프라인 이으기) 이것만 하면 여기 구간은 끝!